# Wrangling data summative

In [ ]:
#Library imports
import bs4 as bs
import pprint as pp
import pandas as pd 
import re
import mwparserfromhell as mwp

import tldextract
import urllib.request

#### Part 1: Get the data's

# Notes: 
# - Use page id in case countries change name (Czechia)
# - (Nuisance of the XML tag called text)
# - Keep two years separate
# - keep "/n" returns in -- because 

# Function to get XML from files, iterate through pages and pull out id,title (country name) 
# and text into a pandas DataFrame object

def makeDF(date):
    wikitext = open("WD_wikipediaCountries_{}.xml".format(date),"rb").read().decode("utf-8")   
    wikisoup = bs.BeautifulSoup(wikitext, "lxml")
    rows = []
    for c,i in enumerate(wikisoup.findAll("page")):
        newdic = {}                                            
        newdic["country"] = i.title.text
        newdic["text{}".format(date[4:])] = i.find("text").text
        temp_df  = pd.DataFrame([newdic.values()],index=[i.id.text] ,columns=newdic.keys())
        rows.append(temp_df)
    wiki_df = pd.concat(rows,axis=0)
    return wiki_df

df_2009 = makeDF("01012009")
df_2019 = makeDF("07012019")

mergedFrame = df_2009.merge(df_2019, left_index=True,right_index=True)

#Check nothing has gone wrong - that both columns are aligned
display(mergedFrame[mergedFrame["country_x"] != mergedFrame["country_y"]])

#delete "country_y" and rename "country_x" to country
mergedFrame = mergedFrame.drop("country_y",axis=1)
mergedFrame.columns = ["country","text2009","text2019"]

bothyears = ["2009","2019"] 

#### Part 2 - cleaning

### Part 2.1 - Extracting links

## extract the links from a text
def what_links(dirty_text, year, istest=False):
    ustlds = ["gov","edu","mil"]
    if istest == True:
        #Validating the external link extraction
        ext_link_regex = re.compile(r'https?://[\w\./?&=%]*')
        ext_links = ext_link_regex.findall(dirty_text)
    else:
        wikipage = mwp.parse(dirty_text,skip_style_tags=True)
        ext_links = wikipage.filter_external_links()  
    
    counts = {"us_count{}".format(year):0,"other_count{}".format(year):0}
    us_suffixes = []
    other_suffixes = []
    for link in ext_links:
        if istest == True:
            url = link
        else:
            url = link.split(" ",1)[0]
            url = url.replace("[","")
            url = url.replace("]","")
        suffix = tldextract.extract(url).suffix
        if suffix in ustlds:
            counts["us_count{}".format(year)] += 1
            us_suffixes.append(suffix)
        else:
            counts["other_count{}".format(year)] += 1
            other_suffixes.append(suffix)
    counts["us_suffixes{}".format(year)] = set(us_suffixes)
    counts["other_suffixes{}".format(year)] = set(other_suffixes)
    return counts

## extract links from a dataframe
def getLinks(df, years = bothyears,istest = False):
    if istest == True:
        test = "test"
    else:
        test=""
    for year in years:
        df["what_links{}{}".format(year,test)] = df.apply(lambda x: what_links(x["text{}".format(year)],year,istest=istest),axis=1)
        #Unpack the columns
        link_values = {"links_us{}{}".format(year,test):'us_count{}'.format(year),
                       "links_other{}{}".format(year,test):'other_count{}'.format(year),
                       "links_suffix_us{}{}".format(year,test):'us_suffixes{}'.format(year),
                       "links_suffix_other{}{}".format(year,test):'other_suffixes{}'.format(year)}
        for col_name,dic_name in link_values.items():
            df[col_name] = df["what_links{}{}".format(year,test)].map(lambda x: x[dic_name])
        del df["what_links{}{}".format(year,test)]
    return df

#Run the function
mergedFrame = getLinks(mergedFrame)

for year in bothyears:
    mergedFrame["links_total{}".format(year)] = mergedFrame.apply(lambda x: x["links_own{}".format(year)] + x["links_us{}".format(year)] + x["links_other{}".format(year)],axis=1)

## Tests on regex vs mwp
mergedFrame = getLinks(mergedFrame, istest=True)

mean_2009_own = (mergedFrame["links_own2009"] - mergedFrame["links_own2009test"]).mean()
mean_2009_us = (mergedFrame["links_us2009"] - mergedFrame["links_us2009test"]).mean()

mean_2019_own = (mergedFrame["links_own2019"] - mergedFrame["links_own2019test"]).mean()
mean_2019_us = (mergedFrame["links_us2019"] - mergedFrame["links_us2019test"]).mean()

print("2009 own: {}".format(mean_2009_own))
print("2009 US: {}".format(mean_2009_us))
print("2019 own: {}".format(mean_2019_own))
print("2019 US: {}".format(mean_2019_us))

# test_cols = [title for title in mergedFrame.columns if "test" in title]
# for title in test_cols:
#     del mergedFrame[title]

### Part 2.2 - Getting article length in number of sentances

# Steps/process:
# - What we want to tidy up is any place where standardised by Wikipedia
# - With wiki parser
#  - Remove internal links which are lists, categories or languages BUT not Images (as contain text)
#  - Remove navigation elements (templates) as same across categories BUT not columns (as contain text)
#  - Remove tags as these these were not handled effectively
#  - Split Image links manually (as wmparserfromhell has issues)
#  - Clean out the columns so that we just get text
#  - automated get display text on page -- removal of all other elements (external links, references, html elements etc)

### Define some functions to clean up the data

## Define what needs to be removed
def getCleaning(wikicode):
    templates = wikicode.filter_templates() 
    templates = [template for template in templates if "column" not in template]
    
    tags = wikicode.filter_tags() #remove <ref></ref>

    int_links = wikicode.filter_wikilinks()
    int_links_bad = [link for link in int_links if ':' in link and 'Image' not in link] #Remove everything thats not an image
    int_links_bad +=  [link for link in int_links if 'List' in link] #remove links to lists
    
    to_clean = templates +int_links_bad + tags
    return to_clean

## Get the text from columns and images then strip everything 
def tidyPage(clean_wikicode):
    new_int_links = clean_wikicode.filter_wikilinks()      #clean the links
    new_int_links = set([str(link) for link in new_int_links])
    for link in new_int_links:
        if "Image" in link:   #get the display text out of the image wrapper
            splitimage = link.split("|")
            imagetext = splitimage[len(splitimage)-1]
            imagetext = re.sub("]]$","",imagetext).strip()
            try:
                clean_wikicode.replace(link,str(imagetext))
            except:
                pass
                print("Error with image: {}".format(imagetext))     #Catches images with no text

    new_templates = clean_wikicode.filter_templates()
    for column in new_templates:  #get the text out of the columns in a table wrapper
        col = re.sub("\n","",str(column))
        splitcols = col.split("|col")
        splitcols = splitcols[1:]
        splitcols = [col.split("=",1)[1] for col in splitcols]
        colphrase = ' '.join(splitcols)
        try:
            clean_wikicode.replace(str(column),splitcols)
        except:
            pass
            print("Error in columns")
    
    output_code = clean_wikicode.strip_code()
    return output_code

## Run all of this to clear out the gubbins
def cleanPage(page):
    wikipage = mwp.parse(page,skip_style_tags=True)
    obj_to_remove = getCleaning(wikipage)   
    
    for item in obj_to_remove:
        try:
            wikipage.remove(item)
        except:
            pass #when item has already been removed
        
    clean_wikicode = tidyPage(wikipage)  
    return clean_wikicode

#Apply cleaning
mergedFrame["clean_text2019"] = mergedFrame["text2019"].map(lambda x: cleanPage(x)) 
mergedFrame["clean_text2009"] = mergedFrame["text2009"].map(lambda x: cleanPage(x))

##remove numbers with decimal place in between
for year in bothyears:
    mergedFrame['clean_text{}'.format(year)] = mergedFrame['clean_text{}'.format(year)].map(lambda x: re.sub("[0-9]\.[0-9]",",",x))

##get number of sentances 
for year in bothyears:
    mergedFrame['sent_length{}'.format(year)] = mergedFrame['clean_text2{}'.format(year)].map(lambda x: len(re.compile(r"[A-Z][^\.!?]*[\.!?]").findall(x)))

## Test sentence regex on random articles
import random
rand_articles = [random.randint(1,196) for x in range(10)]

for c,art in enumerate(rand_articles):
    if c % 2 == 0:
        year = "2009"
    else:
        year = "2019"
    display(mergedFrame["clean_text{}".format(year)][art])
    
### Part 2.3 - prepping for analysis

#Exclude short articles and US
mergedFrame["exclude"] = ((mergedFrame["links_us2009"] == 0) & (mergedFrame["sent_length2009"] < 10)) | (mergedFrame["country"]=="United States")

mergedFrame.to_csv('mergedFrame.csv')

#### Part 3 - analysis

#new libraries
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline
from scipy import stats

#Open CSV
merged_df = pd.read_csv('mergedFrame.csv',index_col=0)
# display(mergedFrame.head())

valid_merged = merged_df[merged_df["exclude"]==False]
# print(len(valid_merged))

### Part 3.1 - link analysis
#link descriptives
link_desc_df = valid_merged[[name for name in valid_merged.columns if "links_us" in name]].describe()
display(link_desc_df)
link_desc_df.to_csv("link_descriptives.csv")

#link t-test
link_ttest = stats.ttest_rel(valid_merged["links_us2019"],valid_merged["links_us2009"])
link_ttest_outputs = [{"type":"link","test":link_ttest[0],"pvalue":link_ttest[1]}]
link_ttest_df = pd.DataFrame(link_ttest_outputs)
link_ttest_df.to_csv("link_ttest.csv")
display(link_ttest_df)

#link plot
link_plot = sns.distplot(valid_merged["links_us2009"],color="red",label="2009")
# plt.show()
sns.distplot(valid_merged["links_us2019"],color="blue",label="2019")
# plt.show()

link_plot.set_title("Number of links to US top-level domains")
link_plot.set(xlabel="Links to US sources",ylabel="Density")
link_plot.legend()
# plt.xlim(right= 0.035)
fig1 = link_plot.get_figure()
fig1.savefig(fname ="link_plot.png",dpi =500)

plt.show()

### Part 3.2 - sentence analysis

#article descriptives
art_desc_df = valid_merged[[name for name in valid_merged.columns if "sent_length" in name]].describe()
display(art_desc_df)
art_desc_df.to_csv("art_descriptives.csv")

#article t-test
art_ttest = stats.ttest_rel(valid_merged["sent_length2019"],valid_merged["sent_length2009"])
art_ttest_outputs = [{"type":"sent length","test":art_ttest[0],"pvalue":art_ttest[1]}]
art_ttest_df = pd.DataFrame(art_ttest_outputs)
art_ttest_df.to_csv("art_ttest_outputs.csv")
display(art_ttest_df)

#plot for article
art_plot = sns.distplot(valid_merged["sent_length2009"],color="red",label="2009") #,kde=False
sns.distplot(valid_merged["sent_length2019"],color="blue",label="2019")

art_plot.set_title("Number of sentences per article")
art_plot.set(xlabel="Number of sentences",ylabel="Density")
art_plot.legend()
fig1 = art_plot.get_figure()
fig1.savefig(fname ="art_plot.png",dpi =500)

plt.show()

#article correlation
bothyears = ["2009","2019"]
# bothlinks = ["links_us"] #"links_own",
correl_list = []

for year in bothyears:
    tempdic = {}
    pearson = stats.pearsonr(x = valid_merged["links_us{}".format(year)], y = valid_merged["sent_length{}".format(year)])
    tempdic["pearson_test"] = pearson[0]
    tempdic["pearson_pvalue"] = pearson[1]
    temp_df = pd.DataFrame([tempdic.values()],index=[year] ,columns=tempdic.keys())
    correl_list.append(temp_df)
    sns.jointplot(x = valid_merged["links_us{}".format(year)], y = valid_merged["sent_length{}".format(year)])
    plt.show()
correl_link_df = pd.concat(correl_list,axis=0)
correl_link_df.to_csv("art_correlation.csv")
display(correl_link_df)

### Part 3.3 - link per sentence analysis

#links per sentence columns
bothyears = ["2009","2019"]
for year in bothyears:
    valid_merged["lps_us{}".format(year)] = valid_merged.apply(lambda x: x.loc["links_us{}".format(year)]/x.loc["sent_length{}".format(year)],axis=1)

#descriptives for lps
lps_desc_df = valid_merged[[name for name in valid_merged.columns if "lps_us" in name]].describe()
display(lps_desc_df)
lps_desc_df.to_csv("lps_descriptives.csv")

#t-test for lps
lps_ttest = stats.ttest_rel(valid_merged["lps_us2019"],valid_merged["lps_us2009"])
lps_ttest_outputs = [{"type":"lps","test":lps_ttest[0],"pvalue":lps_ttest[1]}]
lps_ttest_df = pd.DataFrame(lps_ttest_outputs)

lps_ttest_df.to_csv("lps_ttest_outputs.csv")
display(lps_ttest_df)

#plot for lps
lps_plot = sns.distplot(valid_merged["lps_us2009"],color="red",label="2009") #,kde=False
sns.distplot(valid_merged["lps_us2019"],color="blue",label="2019")

lps_plot.set_title("Links to US top-level domains per sentence in article")
lps_plot.set(xlabel="Links to US sources per sentence in article",ylabel="Density")
lps_plot.legend()
plt.ylim(top= 37)
fig1 = lps_plot.get_figure()
fig1.savefig(fname ="lps_plot.png",dpi =500)

plt.show()